# Table of Contents
1. [download/read csv file](#sec1)

In [ ]:
#packages
%pip install numpy
%pip install pandas
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
     - -------------------------------------- 0.4/11.6 MB 8.1 MB/s eta 0:00:02
     ---- ----------------------------------- 1.2/11.6 MB 12.4 MB/s eta 0:00:01
     ------- -------------------------------- 2.2/11.6 MB 15.8 MB/s eta 0:00:01
     ---------- ----------------------------- 3.2/11.6 MB 16.9 MB/s eta 0:00:01
     ------------- -------------------------- 3.9/11.6 MB 17.8 MB/s eta 0:00:01
     ---------------- ----------------------- 4.8/11.6 MB 17.1 MB/s eta 0:00:01
     ------------------- -------------------- 5.6/11.6 MB 17.2 MB/s eta 0:00:01
     ---------------------- ----------------- 6.5/11.6 MB 17.3 MB/s eta 0:00:01
     ------------------------- -------------- 7.4/11.6 MB 18.2 MB/s eta 0:00:01
     ----------------------------- ---------- 8.4/11.6 MB 18.6 MB/s eta 0:00:01
     ------------------------------- -------- 9.1/11.6 MB 18.8 MB/s eta 0:00:01
     --------------------------------- ----- 10.1


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<a id = "sec1"></a>
## **Download/Read CSV file**

In [ ]:
# read csv
data_path = 'C:\DSB\PIT-UN-Project4\MODIFIED_dc_md_va_flash_floods_1996_pres.csv'
df = pd.read_csv(data_path)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,agricultural_damage,animal_loss,nor_easter,thunderstorm,hurricane,tornado,lightning,mudslide,campground_damage,DATA_SOURCE
0,199601,19,1100,199601,19,1300,2403644,5541358,DISTRICT OF COLUMBIA,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDC
1,199606,18,30,199606,18,200,1014286,5561204,DISTRICT OF COLUMBIA,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDC
2,199606,20,2200,199606,20,2300,2040906,5561207,DISTRICT OF COLUMBIA,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDC
3,199609,6,1200,199609,6,1800,2049837,5572405,DISTRICT OF COLUMBIA,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDC
4,199611,8,1600,199611,8,1715,2049872,5572790,DISTRICT OF COLUMBIA,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDC


In [ ]:
#list of OG tags
['death','injury','evacuation','rescue','car_crash','home_damage','infrastructure_damage','school_damage','hospital_damage','road_closure','power_outage']

In [ ]:
#populate columns for OG tags
df['death'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'death' in x else 0)
df['injury'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'injury' in x else 0)
df['evacuation'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'evacuation' in x else 0)
df['rescue'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'rescue' in x else 0)
df['car_crash'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'car_crash' in x else 0)
df['home_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'home_damage' in x else 0)
df['infrastructure_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'infrastructure_damage' in x else 0)
df['school_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'school_damage' in x else 0)
df['hospital_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'hospital_damage' in x else 0)
df['road_closure'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'road_closure' in x else 0)
df['power_outage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'power_outage' in x else 0)
df['tree_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'tree_damage' in x else 0)
df['vehicle_loss'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'vehicle_loss' in x else 0)
df['agricultural_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'animal_loss' in x else 0)
df['campground_damage'] = df['IMPACT_PREDICTORS'].apply(str).apply(lambda x: 1 if 'tree_damage' in x else 0)

#weather events
df['thunderstorm'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'thunderstorm' in x else 0)
df['hurricane'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'hurricane' in x else 0)
df['tornado'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'thunderstorm' in x else 0)
df['lightning'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'thunderstorm' in x else 0)
df['mudslide'] = df['weather_events'].apply(str).apply(lambda x: 1 if 'mudslide' in x else 0)

In [12]:
#check columns
df[['death','injury','evacuation','rescue','car_crash','home_damage','infrastructure_damage','school_damage','hospital_damage','road_closure','power_outage']].head()

,death,injury,evacuation,rescue,car_crash,home_damage,infrastructure_damage,school_damage,hospital_damage,road_closure,power_outage
0,0,0,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,1,0,0,1,0
